In [1]:
from __future__ import print_function

# Import MNIST data
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

In [2]:
import numpy as np
data_X = np.zeros((100000,2))
data_y = np.zeros((100000,1))
import random

for i in range(data_X.shape[0]):
    data_X[i,0] = random.uniform(0, 1)
    data_X[i,1] = random.uniform(0, 1)
    if(data_X[i,0] >= 0.5 and data_X[i,1]>=0.5):
        data_y[i] = 1
    else:
        data_y[i] = 0
        
testx = np.zeros((100,2))
testy = np.zeros((100,1))
import random

for i in range(testx.shape[0]):
    testx[i,0] = random.uniform(0, 1)
    testx[i,1] = random.uniform(0, 1)
    if(testx[i,0] >= 0.5 and testx[i,1]>=0.5):
        testy[i] = 1
    else:
        testy[i] = 0


In [2]:
import numpy as np
data = np.loadtxt("Skin_NonSkin.txt")

In [4]:
import random

In [84]:
X = data[:,:3]/255.0
y = data[:,3]
y[y==2] = 0
index = random.sample(range(X.shape[0]),X.shape[0])
X = X[index]
y = y[index]
X_train = X[:int(X.shape[0]*70/100.0)]
y_train = y[:int(X.shape[0]*70/100.0)]
X_test = X[int(X.shape[0]*70/100.0):]
y_test = y[int(X.shape[0]*70/100.0):]
print(X_train.shape)
print(y_train.shape)

(171539, 3)
(171539,)


In [85]:
print(np.min(X_train))

0.0


In [86]:
import numpy as np
# Parameters
learning_rate = 0.001
num_steps = 10000
batch_size = 2000
display_step = 1000

# Network Parameters
n_hidden_1 = 2 #1st layer number of neurons
n_hidden_2 = 15 #2nd layer number of neurons
num_input = 3 # MNIST data input (img shape: 28*28)
num_classes = 1 # MNIST total classes (0-9 digits)

# initialize intervals for via
# [0,1] since sigmoid is used
#input_interval = np.zeros((2,num_input))
#input_interval[1,:] = input_interval[1,:] + 1
#input_interval[1,0] = 1
#input_interval[1,1] = 0.4
"""
input_interval = []
for i in range(num_input):
    input_interval.append([(0,255)])
input_interval = np.array(input_interval)
"""

#h1_interval = np.zeros((2,n_hidden_1))
#h1_interval[1,:] = h1_interval[1,:] + 1
"""
for i in range(n_hidden_1):
    h1_interval.append([0,1])
h1_interval = np.array(h1_interval)  
"""
#h2_interval = np.zeros((2,n_hidden_2))
#h2_interval[1,:] = h2_interval[1,:] + 1
"""
h2_interval = []
for i in range(n_hidden_2):
    h2_interval.append([(0,1)])
h2_interval = np.array(h2_interval)
"""

#out_interval = np.zeros((2,num_classes))
#out_interval[1,:] = out_interval[1,:] + 1
"""
out_interval = []
for i in range(num_classes):
    out_interval.append([(0,1)])
out_interval = np.array(out_interval)
"""

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [87]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    #'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    #'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [88]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.sigmoid(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    # Hidden fully connected layer with 256 neurons
    #layer_2 = tf.sigmoid(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    # Output fully connected layer with a neuron for each class
    out_layer = tf.sigmoid(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

In [89]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
loss = -(Y * tf.log(logits) + (1 - Y) * tf.log( 1 - logits))
loss_op = tf.reduce_mean(tf.reduce_sum(loss, reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
#correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
correct_pred = tf.equal(tf.greater_equal(logits, 0.5), tf.greater_equal(Y, 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [90]:
import random
def next_batch(batch_size):
    index = random.sample(range(X_train.shape[0]), batch_size)
    return X_train[index],y_train[index].reshape(-1,1)

In [91]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    logit,f_accuracy,f_weights,f_biases = sess.run([logits,accuracy,weights,biases], feed_dict={X: X_test,Y: y_test.reshape(-1,1)})
    print("Testing Accuracy:", f_accuracy)

Step 1, Minibatch Loss= 0.5728, Training Accuracy= 0.811
Step 1000, Minibatch Loss= 0.4616, Training Accuracy= 0.802
Step 2000, Minibatch Loss= 0.3815, Training Accuracy= 0.789
Step 3000, Minibatch Loss= 0.2857, Training Accuracy= 0.814
Step 4000, Minibatch Loss= 0.2421, Training Accuracy= 0.906
Step 5000, Minibatch Loss= 0.2017, Training Accuracy= 0.930
Step 6000, Minibatch Loss= 0.1740, Training Accuracy= 0.938
Step 7000, Minibatch Loss= 0.1555, Training Accuracy= 0.947
Step 8000, Minibatch Loss= 0.1266, Training Accuracy= 0.961
Step 9000, Minibatch Loss= 0.1318, Training Accuracy= 0.951
Step 10000, Minibatch Loss= 0.1297, Training Accuracy= 0.950
Optimization Finished!
Testing Accuracy: 0.95866317


In [92]:
logit = np.squeeze(logit)
y_test[logit>0.5] = 1
y_test[logit<=0.5] = 0

In [109]:
for i in range(X_test.shape[0]):
    if(y_test[i]==1):
        print(X_test[i])

[0.38823529 0.59215686 0.87058824]
[0.56862745 0.70980392 0.96078431]
[0.23529412 0.4745098  0.7254902 ]
[0.43921569 0.59607843 0.82352941]
[0.54117647 0.65490196 0.87843137]
[0.65490196 0.70980392 0.89803922]
[0.18431373 0.33333333 0.54509804]
[0.46666667 0.61568627 0.82745098]
[0.40392157 0.58039216 0.87058824]
[0.48235294 0.60392157 0.8745098 ]
[0.21568627 0.4        0.61176471]
[0.68235294 0.75294118 0.96078431]
[0.40784314 0.54509804 0.79607843]
[0.51372549 0.62352941 0.83529412]
[0.52156863 0.63137255 0.88235294]
[0.69019608 0.76078431 0.96862745]
[0.44313725 0.65098039 0.91372549]
[0.2        0.38431373 0.59607843]
[0.59215686 0.75686275 0.98823529]
[0.61960784 0.69803922 0.91372549]
[0.53333333 0.69803922 0.95294118]
[0.48235294 0.62352941 0.8745098 ]
[0.40392157 0.56470588 0.81176471]
[0.63529412 0.76862745 1.        ]
[0.32941176 0.36078431 0.51764706]
[0.79215686 0.83137255 0.98823529]
[0.58431373 0.57254902 0.79215686]
[0. 1. 1.]
[0.25882353 0.45882353 0.70588235]
[0.588235

[0.60392157 0.74509804 0.99607843]
[0.25490196 0.51764706 0.78823529]
[0.37254902 0.56862745 0.86666667]
[0.57254902 0.61960784 0.87058824]
[0.24705882 0.49803922 0.75294118]
[0.4745098  0.6        0.83137255]
[0.43137255 0.6        0.84705882]
[0. 1. 1.]
[0.49019608 0.50196078 0.71764706]
[0.6        0.70588235 0.90196078]
[0.44313725 0.59607843 0.83921569]
[0.72941176 0.79215686 0.96078431]
[0.3372549  0.50980392 0.76078431]
[0.43137255 0.47058824 0.89411765]
[0.64313725 0.77647059 0.91764706]
[0.25490196 0.49019608 0.76470588]
[0.35294118 0.26666667 0.96862745]
[0.61960784 0.75686275 1.        ]
[0.64705882 0.69803922 0.90196078]
[0.47843137 0.45098039 0.62745098]
[0.37647059 0.50196078 0.76078431]
[0.21568627 0.39607843 0.58431373]
[0.59607843 0.74901961 0.99215686]
[0.18823529 0.41176471 0.61568627]
[0.45098039 0.58823529 0.83921569]
[0.67058824 0.76470588 0.94509804]
[0.50588235 0.65098039 0.87058824]
[0.56470588 0.73333333 0.99607843]
[0.24313725 0.33333333 0.52156863]
[0.447058

[0.56078431 0.71764706 0.97254902]
[0.59607843 0.66666667 0.90588235]
[0.65490196 0.7372549  0.94117647]
[0.48627451 0.61960784 0.85490196]
[0.38823529 0.50196078 0.74117647]
[0.78431373 0.81568627 1.        ]
[0.52156863 0.63921569 0.85490196]
[0.38039216 0.36078431 0.6       ]
[0.47058824 0.59607843 0.85098039]
[0.60392157 0.65098039 0.90196078]
[0.4745098  0.61568627 0.86666667]
[0.32156863 0.4745098  0.74509804]
[0.43921569 0.59607843 0.82352941]
[0.2627451  0.49803922 0.73333333]
[0.54509804 0.65098039 0.90196078]
[0.61568627 0.75294118 0.90980392]
[0.6627451  0.72156863 0.91372549]
[0.80784314 0.85098039 1.        ]
[0.65490196 0.70980392 0.91372549]
[0.2        0.3372549  0.53333333]
[0.39215686 0.53333333 0.74509804]
[0.47058824 0.62745098 0.83137255]
[0.34509804 0.54901961 0.84705882]
[0.43137255 0.5372549  0.75686275]
[0.5372549  0.69803922 0.89019608]
[0.12156863 0.71764706 0.81176471]
[0.58039216 0.69411765 0.87058824]
[0.2        0.36470588 0.54901961]
[0.52156863 0.639215

[0.46666667 0.61176471 0.83137255]
[0.19215686 0.29019608 0.54117647]
[0.17254902 0.36470588 0.55294118]
[0.68235294 0.73333333 0.9372549 ]
[0.45882353 0.58823529 0.85882353]
[0.38431373 0.35686275 0.59607843]
[0.29803922 0.42352941 0.68235294]
[0.48235294 0.62745098 0.8627451 ]
[0.21568627 0.44313725 0.69803922]
[0.36078431 0.53333333 0.85882353]
[0.68235294 0.7372549  0.92941176]
[0.35686275 0.38431373 0.57647059]
[0.61176471 0.66666667 0.85882353]
[0.4627451  0.61176471 0.8627451 ]
[0.2627451  0.50196078 0.75294118]
[0.43137255 0.6        0.82352941]
[0.60784314 0.71372549 0.93333333]
[0.47843137 0.61960784 0.83137255]
[0.54901961 0.69019608 0.87843137]
[0.12941176 0.46666667 0.34901961]
[0.40392157 0.55294118 0.78039216]
[0.24705882 0.36078431 0.51372549]
[0.61568627 0.76078431 1.        ]
[0.48627451 0.62352941 0.8745098 ]
[0.59215686 0.72156863 0.8745098 ]
[0.21176471 0.38039216 0.57254902]
[0.4        0.52156863 0.72156863]
[0.46666667 0.63529412 0.83529412]
[0.40392157 0.549019

[0.2627451  0.45098039 0.67843137]
[0.05490196 0.75686275 0.99607843]
[0.54901961 0.65098039 0.90980392]
[0.40784314 0.58823529 0.86666667]
[0.58823529 0.73333333 1.        ]
[0.25490196 0.50588235 0.75686275]
[0.58823529 0.63137255 0.89803922]
[0.45882353 0.61568627 0.84705882]
[0.58823529 0.7372549  0.98823529]
[0.22352941 0.43921569 1.        ]
[0.5372549  0.67843137 0.85882353]
[0.61960784 0.75686275 1.        ]
[0.66666667 0.72941176 0.90980392]
[0.45882353 0.57254902 0.8627451 ]
[0.31372549 0.36078431 0.51764706]
[0.59215686 0.74117647 0.99215686]
[0.24705882 0.47843137 0.74901961]
[0.37254902 0.56470588 0.85490196]
[0.39215686 0.59215686 0.86666667]
[0.2        0.38431373 0.58431373]
[0.16862745 0.35686275 0.58431373]
[0.44705882 0.84705882 0.82745098]
[0.38431373 0.50588235 0.74509804]
[0.67843137 0.7372549  0.91764706]
[0.36862745 0.53333333 0.85882353]
[0.36078431 0.38431373 0.59215686]
[0.19215686 0.41960784 0.65098039]
[0.54509804 0.68235294 0.87843137]
[0.31372549 0.698039

...

[0.19607843 0.28235294 0.50980392]
[0.59607843 0.75686275 1.        ]
[0.24313725 0.41960784 0.61960784]
[0.51372549 0.62745098 0.89019608]
[0.         0.52941176 0.40392157]
[0.48235294 0.48627451 0.70588235]
[0.41176471 0.55294118 0.82745098]
[0.21568627 0.5254902  0.44313725]
[0.4745098  0.56862745 0.63921569]
[0.47843137 0.62745098 0.87843137]
[0.51372549 0.65490196 0.89019608]
[0.40784314 0.42352941 0.65098039]
[0.22352941 0.41176471 0.6627451 ]
[0.50196078 0.64705882 0.85098039]
[0.4        0.80392157 0.78431373]
[0.30588235 0.37254902 0.57254902]
[0.77647059 0.81960784 0.99215686]
[0.21960784 0.40392157 0.61568627]
[0.63529412 0.71372549 0.92941176]
[0.         0.41960784 0.97647059]
[0.58431373 0.72941176 0.97254902]
[0.47058824 0.62745098 0.88235294]
[0.25490196 0.41176471 0.61960784]
[0.39607843 0.56078431 0.79215686]
[0.43529412 0.59607843 0.84313725]
[0.23921569 0.41568627 0.67843137]
[0.35686275 0.27058824 0.97254902]
[0.21176471 0.36862745 0.57254902]
[0.31372549 0.352941

[0.20392157 0.44705882 1.        ]
[0.45490196 0.56078431 0.66666667]
[0.42745098 0.64705882 0.87843137]
[0.21568627 0.41176471 0.61960784]
[0.4745098  0.62352941 0.8745098 ]
[0.34117647 0.42352941 0.57254902]
[0.24705882 0.4745098  0.70588235]
[0.40392157 0.59215686 0.8745098 ]
[0.40392157 0.58431373 0.8627451 ]
[0.43137255 0.6        0.82352941]
[0.65882353 0.71372549 0.91764706]
[0.62745098 0.70588235 0.89411765]
[0.22745098 0.4745098  0.74117647]
[0.40392157 0.42745098 0.63529412]
[0.38823529 0.57647059 0.85882353]
[0.5372549  0.71372549 0.97647059]
[0.2       0.4745098 0.7372549]
[0.48627451 0.62745098 0.8627451 ]
[0.23529412 0.3254902  0.51372549]
[0.50588235 0.63137255 0.89019608]
[0.23529412 0.43921569 0.64705882]
[0.65882353 0.72941176 0.89019608]
[0.34901961 0.25490196 0.95294118]
[0.58823529 0.7372549  0.98823529]
[0. 1. 1.]
[0.42352941 0.6        0.82352941]
[0.61960784 0.76470588 1.        ]
[0.53333333 0.69019608 0.87843137]
[0.56078431 0.6627451  0.9254902 ]
[0.57254902 

[0.60784314 0.65490196 0.90588235]
[0.47058824 0.56470588 0.82352941]
[0.56078431 0.6745098  0.89803922]
[0.39215686 0.58039216 0.8627451 ]
[0.45882353 0.63137255 0.88235294]
[0.21176471 0.47843137 0.74117647]
[0.61176471 0.68627451 0.85490196]
[0.51372549 0.51764706 0.7372549 ]
[0.49019608 0.62352941 0.88235294]
[0.30980392 0.70196078 0.91764706]
[0.43529412 0.58431373 0.81176471]
[0.32941176 0.55686275 0.81176471]
[0.39607843 0.57647059 0.87843137]
[0.45098039 0.56470588 0.85490196]
[0.6        0.74901961 1.        ]
[0.42745098 0.56078431 0.79607843]
[0.38823529 0.55294118 0.81568627]
[0.51372549 0.63137255 0.90980392]
[0.58039216 0.74509804 1.        ]
[0.49019608 0.50588235 0.73333333]
[0.65882353 0.71372549 0.92156863]
[0.6        0.7372549  0.87058824]
[0.76862745 0.81960784 1.        ]
[0.61568627 0.68627451 0.91764706]
[0.36470588 0.50980392 0.77647059]
[0.46666667 0.60392157 0.85098039]
[0.79215686 0.82745098 0.99607843]
[0.58431373 0.73333333 0.98431373]
[0.61960784 0.686274

[0.44313725 0.55686275 0.85098039]
[0.00784314 0.5372549  0.41176471]
[0.58431373 0.75686275 0.99607843]
[0.65098039 0.71372549 0.88235294]
[0.38431373 0.35686275 0.59607843]
[0.49019608 0.66666667 0.86666667]
[0.49019608 0.61568627 0.87058824]
[0.4        0.54509804 0.77254902]
[0.45882353 0.59607843 0.84705882]
[0.53333333 0.69803922 0.95294118]
[0.5372549  0.64313725 0.89411765]
[0.58431373 0.73333333 0.98431373]
[0.2745098  0.49803922 0.76862745]
[0.18039216 0.4        0.61568627]
[0.22352941 0.34509804 0.55294118]
[0.56862745 0.72941176 0.97647059]
[0.44705882 0.46666667 0.68235294]
[0.36470588 0.58039216 0.8745098 ]
[0.22352941 0.4627451  0.72941176]
[0.         0.5372549  0.41568627]
[0.24313725 0.43529412 0.61568627]
[0.41176471 0.60392157 0.87058824]
[0.61960784 0.70196078 0.94509804]
[0.4745098  0.61568627 0.85098039]
[0.4745098  0.60392157 0.8745098 ]
[0.51372549 0.66666667 0.87058824]
[0.41960784 0.81568627 0.75686275]
[0.17254902 0.37254902 0.61960784]
[0.54901961 0.654901

[0.54117647 0.52941176 0.96078431]
[0.65490196 0.70980392 0.91372549]
[0.69019608 0.75294118 0.93333333]
[0.40392157 0.54901961 0.81568627]
[0.77254902 0.81568627 0.98823529]
[0.65882353 0.72156863 0.90196078]
[0.65098039 0.71372549 0.89411765]
[0.6        0.74901961 1.        ]
[0.39215686 0.50980392 0.76470588]
[0.25098039 0.51764706 0.78039216]
[0.24313725 0.48235294 0.74117647]
[0.51764706 0.66666667 0.87843137]
[0.6745098  0.74117647 0.90980392]
[0.23921569 0.45098039 0.68627451]
[0.57647059 0.73333333 0.98823529]
[0.14509804 0.34901961 0.55686275]
[0.23529412 0.42352941 0.6745098 ]
[0.63529412 0.67058824 0.89411765]
[0.53333333 0.67058824 0.86666667]
[0.54901961 0.70980392 0.99607843]
[0.18431373 0.29019608 0.51372549]
[0.6627451  0.71372549 0.93333333]
[0.10196078 0.8        0.96078431]
[0.68235294 0.74509804 0.9254902 ]
[0.50980392 0.60784314 0.80392157]
[0.36078431 0.50980392 0.76470588]
[0.37647059 0.56078431 0.77254902]
[0.16470588 0.35686275 0.55294118]
[0.54901961 0.701960

In [95]:
import math

def sigmoid(x):
    if(x>50):
        return 1
    if(x<-50):
        return 1e-12
    return 1 / (1 + math.exp(-x))

In [96]:
import numpy as np
h1 = np.array(f_weights['h1'])
#h2 = np.array(f_weights['h2'])
out_w = np.array(f_weights['out'])
b1 = np.array(f_biases['b1'])
#b2 = np.array(f_biases['b2'])
out_b = np.array(f_biases['out'])

In [97]:
h1

array([[ 0.06192115,  7.0181336 ],
       [-7.8160367 ,  3.1022456 ],
       [-0.06261314, -8.99723   ]], dtype=float32)

In [115]:
from scipy.optimize import minimize
def objective(x,w):
    w = np.array(w)
    b = w[-1]
    w = w[:-1]
    #k = sigmoid(np.sum(x*w) + b)
    k = np.sum(x*w) + b
    return np.sum(k)
def objective_max(x,w):
    w = np.array(w)
    b = w[-1]
    w = w[:-1]
    #k = sigmoid(np.sum(x*w) + b)
    k = np.sum(x*w) + b
    return -np.sum(k)

In [116]:
# forward phase for via
from scipy import optimize
def forward(input_interval,h1_interval,out_interval):
    for i in range(h1.shape[1]):
        w = h1[:,i]
        #w = np.hstack((h1[:,i],b1[i]))
        
        #w = tuple(np.hstack((w,b1[i])))
        bnds = []
        for j in range(input_interval.shape[1]):
            bnds.append((input_interval[0,j],input_interval[1,j]))
        #x0 = (0,0)
        
        bnds = np.array(bnds)
        x0 = np.zeros(input_interval.shape[1])
        for j in range(x0.shape[0]):
            x0[j] = bnds[0,i]
        bnds = tuple(bnds)
        
        sol = optimize.linprog(c = w,bounds=bnds,method='simplex',options=dict(tol=1e-15))
        print("minimum")
        print(sol['fun'])
        print(b1[i])
        print(sigmoid(sol['fun']+b1[i]))
        h1_interval[0,i] = sigmoid(sol['fun'] + b1[i])
        sol = optimize.linprog(c = -w,bounds=bnds,method='simplex')
        #sol = minimize(objective_max,x0,(w,),bounds = bnds)
        h1_interval[1,i] = sigmoid(-(sol['fun'])+b1[i])
        print("h1 change in forward")
        print(h1_interval)
        
        
    # out layer
    for i in range(out_w.shape[1]):
        w = out_w[:,i]
        #w = np.hstack((h1[:,i],b1[i]))
        #w = tuple(np.hstack((w,out_b[i])))
        bnds = []
        for j in range(h1_interval.shape[1]):
            bnds.append((h1_interval[0,j],h1_interval[1,j]))
        x0 = np.zeros((h1_interval.shape[1]))
        bnds = np.array(bnds)
        bnds = tuple(bnds)
        #sol = minimize (objective,x0,(w,),bounds = bnds)
        sol = optimize.linprog(c = w,bounds=bnds,method='simplex')
        out_interval[0,i] = sigmoid(sol['fun'] + out_b[i])
        #sol = minimize(objective_max,x0,(w,),bounds = bnds)
        sol = optimize.linprog(c = -w,bounds=bnds,method='simplex')
        out_interval[1,i] = sigmoid(-(sol['fun'])+out_b[i])
        print("outchange in forward")
        print(out_interval)
    

In [117]:
def invsig(interval):
    ans = -np.log(1/(interval) - 1 )
    ans[ans<-30] = -30
    return ans

In [118]:
def bound(weights,interval,bias,mode,layer):
    A_ub = weights.T
    ones = np.identity((weights.T).shape[0])
    zeros = np.zeros(((weights.T).shape[0],(weights.T).shape[0]))
    #A_ub = np.hstack((A_ub,ones,zeros))
    # make intervals ready for log
    for i in range(interval.shape[1]):
        if(interval[1,i]>=1):
            interval[1,i]= 1-1e-6
        if(interval[0,i]==0):
            interval[0,i] = 1e-6
            
    b_ub = invsig(interval[1,:]) - bias
    b_ub1 = invsig(interval[0,:]) - bias
    print(b_ub)
    print(b_ub1)
    #A_ub1 = np.hstack((weights.T,zeros,ones))
    A_ub = np.vstack((A_ub,-A_ub))
    b_ub = np.hstack((b_ub,-b_ub1))
    new_interval = np.zeros((A_ub.shape[1]))
    bnds = []
    #print("solving")
    #print(A_ub)
    #print(b_ub)
    if(layer=="hidden"):
        for j in range(A_ub.shape[1]):
            bnds.append((0,1))
    else:
        for j in range(A_ub.shape[1]):
            bnds.append((0,1))
    bnds = tuple(bnds)
    for i in range(A_ub.shape[1]):
        c = np.zeros((A_ub.shape[1]))
        if(mode=="min"):
            c[i] = 1
        else:
            c[i] = -1
        res = scipy.optimize.linprog(c,A_ub = A_ub,b_ub = b_ub,options=dict(tol=1e-15),bounds = bnds)
        #print(res)
        
        new_interval[i] = res['x'][i]
        
    return new_interval

In [119]:
import scipy
def backward(input_interval,h1_interval,out_interval):
    #print("computing for hidden layer")
    max_bounds = bound(out_w,out_interval,out_b,"max","hidden")
    h1_interval[1,:] = max_bounds
    min_bounds = bound(out_w,out_interval,out_b,"min","hidden")
    h1_interval[0,:] = min_bounds
    #print("hidden chagne in backward")
    #print(h1_interval)
    #max_bounds = bound(h2,h2_interval,b2,"max")
    #min_bounds = bound(h2,h2_interval,b2,"min")
    #print("computing for input interval")
    max_bounds = bound(h1,h1_interval,b1,"max","input")
    input_interval[1,:] = max_bounds
    min_bounds = bound(h1,h1_interval,b1,"min","input")
    input_interval[0,:] = min_bounds
    #print(" input interval cahnge in backward")
    #print(input_interval)

In [130]:
def test_consistency(input_interval):
    # initialize hidden layer/s and output intervals
    h1_interval = np.zeros((2,n_hidden_1))
    h1_interval[1,:] = h1_interval[1,:] + 1
    out_interval = np.zeros((2,num_classes))
    out_interval[1,:] = out_interval[1,:] + 1
    abs_diff = 100
    initial_val = np.sum(h1_interval) + np.sum(input_interval) + np.sum(out_interval)
    while(abs_diff>1 ):
        #print("abs diff")
        #print(abs_diff)
        print("input before fwd")
        print("input")
        print(input_interval)
        print(h1_interval)
        print(out_interval)
        forward(input_interval,h1_interval,out_interval)
        print("after fwd")
        print(input_interval)
        print(h1_interval)
        print(out_interval)
        # check for consistency
        if(out_interval[0,0]<0.5):
            print("consistency detected")
            return 1
        print("before bkw")
        print(input_interval)
        print(h1_interval)
        print(out_interval)
        backward(input_interval,h1_interval,out_interval)
        print("after bkw")
        print(input_interval)
        print(h1_interval)
        print(out_interval)
        final_val = np.sum(h1_interval) + np.sum(input_interval) + np.sum(out_interval)
        abs_diff = abs(initial_val - final_val)
        initial_val = final_val
    return 0
        

In [131]:
import copy
# rule extraction using via (and gate)
data_X = X_test
data_y = y_test
input_max = 1
input_min = 0
input_interval = np.zeros((2,num_input))
"""
input_interval[1,0] = 1
input_interval[1,1] = 0.4

h1_interval = np.zeros((2,n_hidden_1))
h1_interval[1,:] = h1_interval[1,:] + 1

# set out interval according to rule to test (rules will be proved via contradiction using inconsistencies in VIA analysis)
out_interval = np.zeros((2,num_classes))
out_interval[1,:] = out_interval[1,:] + 1
"""
#rules = np.zeros((data_X.shape[0],2,num_input))
t = 0
tol = 0.1
#for i in range(data_X.shape[0]):
for i in range(400):
    
    if(data_y[i]==1):
        #print("this time ")
        #print(data_X[i])
        flag = 0
        """
        for j in range(rules.shape[0]):
            if(np.sum(rules[j])==0):
                continue
            if(flag ==1):
                break
            for k in range(data_X.shape[1]):
                if(rules[j,0,k]<data_X[i,k] and rules[j,1,k]>data_X[i,k]):
                    flag = 1
                    break
        """
        if(flag==1):
            continue
        else:
        

            for j in range(input_interval.shape[1]):
                input_interval[0,j] = data_X[i,j]
                input_interval[1,j] = data_X[i,j]
            print("testing this data point")
            print(input_interval)
            
            for k in range(input_interval.shape[1]):
                # try decreasing the lower bound
                test_val = 0
                while(test_val==0):
                    print("try decreasing")
                    print("input interval")
                    print(input_interval)
                    test_interval = np.copy(input_interval)
                    test_interval[0,k] = test_interval[0,k] - tol
                    input_interval = np.copy(test_interval)
                    if(test_interval[0,k]<tol):
                        break
                    test_val = test_consistency(input_interval)
                    if(test_val== 0):
                        input_interval = np.copy(test_interval)
                    else:
                        input_interval[0,k] = input_interval[0,k] + tol
                
                #now try increasing the upper bound
                test_val = 0
                while(test_val==0):
                    print("try increasing")
                    print("input interval")
                    print(input_interval)
                    test_interval = np.copy(input_interval)
                    test_interval[1,k] = test_interval[1,k] + tol
                    input_interval = np.copy(test_interval)
                    if(test_interval[1,k]>1-tol):
                        break
                    test_val = test_consistency(input_interval)
                    if(test_val==0):
                        input_interval = np.copy(test_interval)
                    else:
                        input_interval[1,k] = input_interval[1,k] - tol
            print("rule generate")
            print(input_interval)
            #rules[t] = input_interval
            #t = t+1
            #print(rules)
            



testing this data point
[[0.38823529 0.59215686 0.87058824]
 [0.38823529 0.59215686 0.87058824]]
try decreasing
input interval
[[0.38823529 0.59215686 0.87058824]
 [0.38823529 0.59215686 0.87058824]]
input before fwd
input
[[0.28823529 0.59215686 0.87058824]
 [0.38823529 0.59215686 0.87058824]]
[[0. 0.]
 [1. 1.]]
[[0.]
 [1.]]
minimum
-4.664982178339771
1.6656325
0.04745526056900954
h1 change in forward
[[0.04745526 0.        ]
 [0.04773595 1.        ]]
minimum
-3.972992401497037
-0.64594835
0.009766904837475894
h1 change in forward
[[0.04745526 0.0097669 ]
 [0.04773595 0.01950996]]
outchange in forward
[[0.9002619 ]
 [0.90806224]]
after fwd
[[0.28823529 0.59215686 0.87058824]
 [0.38823529 0.59215686 0.87058824]]
[[0.04745526 0.0097669 ]
 [0.04773595 0.01950996]]
[[0.9002619 ]
 [0.90806224]]
before bkw
[[0.28823529 0.59215686 0.87058824]
 [0.38823529 0.59215686 0.87058824]]
[[0.04745526 0.0097669 ]
 [0.04773595 0.01950996]]
[[0.9002619 ]
 [0.90806224]]
[-0.3753282]
[-0.46539125]
[-0.375

[-3.30000931 -1.457647  ]
[-15.48114204 -13.16956121]
[-3.30000931 -1.457647  ]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.4141992  0.30674145]
 [0.88810052 1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [1.63231663e-01 1.08747863e-01]]
[[0.84280043]
 [0.87649242]]
try increasing
input interval
[[0.03529412 0.4745098  0.7254902 ]
 [0.33529412 0.4745098  0.7254902 ]]
input before fwd
input
[[0.03529412 0.4745098  0.7254902 ]
 [0.43529412 0.4745098  0.7254902 ]]
[[0. 0.]
 [1. 1.]]
[[0.]
 [1.]]
minimum
-3.752025812688996
1.6656325
0.11042636903328874
h1 change in forward
[[0.11042637 0.        ]
 [0.11288301 1.        ]]
minimum
-4.8076570782006955
-0.64594835
0.004262595063000024
h1 change in forward
[[0.11042637 0.0042626 ]
 [0.11288301 0.06621421]]
outchange in forward
[[0.79945573]
 [0.87649242]]
after fwd
[[0.03529412 0.4745098  0.7254902 ]
 [0.43529412 0.4745098  0.7254902 ]]
[[0.11042637 0.0042626 ]
 [0.11288301 0.06621421]]
[[0.79945573]
 [0.87649242]]
before 

[-2.98737845 -1.16720454]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.37420056 0.2607955 ]
 [0.94695968 1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [2.10527957e-01 1.40257502e-01]]
[[0.80113922]
 [0.87391557]]
input before fwd
input
[[0.         0.37420056 0.2607955 ]
 [0.94695968 1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [2.10527957e-01 1.40257502e-01]]
[[0.80113922]
 [0.87391557]]
minimum
-7.8786498457193375
1.6656325
0.001999179285907062
h1 change in forward
[[1.99917929e-03 1.00000000e-06]
 [2.28495636e-01 1.40257502e-01]]
minimum
-7.8363675480489565
-0.64594835
0.00020705564682915964
h1 change in forward
[[1.99917929e-03 2.07055647e-04]
 [2.28495636e-01 9.98837187e-01]]
outchange in forward
[[4.20210123e-04]
 [9.34107778e-01]]
after fwd
[[0.         0.37420056 0.2607955 ]
 [0.94695968 1.         1.        ]]
[[1.99917929e-03 2.07055647e-04]
 [2.28495636e-01 9.98837187e-01]]
[[4.20210123e-04]
 [9.34107778e-01]]
consistency detected
try increas

[-1.96411943 -0.27993579]
[-15.48114204 -13.16956121]
[-1.96411943 -0.27993579]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.24328267 0.11743521]
 [1.         1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [4.25927405e-01 2.83760480e-01]]
[[0.52295749]
 [0.8460075 ]]
try increasing
input interval
[[0.08431373 0.33333333 0.54509804]
 [0.28431373 0.43333333 0.64509804]]
input before fwd
input
[[0.08431373 0.33333333 0.54509804]
 [0.28431373 0.43333333 0.74509804]]
[[0. 0.]
 [1. 1.]]
[[0.]
 [1.]]
minimum
-3.42838136559608
1.6656325
0.1464463962551535
h1 change in forward
[[0.1464464  0.        ]
 [0.27763197 1.        ]]
minimum
-5.078011265922994
-0.64594835
0.003256113273467755
h1 change in forward
[[0.1464464  0.00325611]
 [0.27763197 0.09879708]]
outchange in forward
[[0.52295749]
 [0.85209221]]
after fwd
[[0.08431373 0.33333333 0.54509804]
 [0.28431373 0.43333333 0.74509804]]
[[0.1464464  0.00325611]
 [0.27763197 0.09879708]]
[[0.52295749]
 [0.85209221]]
before bkw

minimum
-4.59063383808323
1.6656325
0.050931404913318414
h1 change in forward
[[0.0509314  0.        ]
 [0.10909168 1.        ]]
minimum
-6.315065686375487
-0.64594835
0.0009472365357068687
h1 change in forward
[[0.0509314  0.00094724]
 [0.10909168 0.14955495]]
outchange in forward
[[0.65699377]
 [0.91286973]]
after fwd
[[0.00392157 0.48039216 0.87058824]
 [0.70392157 0.58039216 0.87058824]]
[[0.0509314  0.00094724]
 [0.10909168 0.14955495]]
[[0.65699377]
 [0.91286973]]
before bkw
[[0.00392157 0.48039216 0.87058824]
 [0.70392157 0.58039216 0.87058824]]
[[0.0509314  0.00094724]
 [0.10909168 0.14955495]]
[[0.65699377]
 [0.91286973]]
[-0.31634042]
[-2.01560334]
[-0.31634042]
[-2.01560334]
[-2.35769146 -0.60674773]
[-15.48114204 -13.16956121]
[-2.35769146 -0.60674773]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.2936371  0.17097316]
 [1.         1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [3.33575201e-01 2.22233784e-01]]
[[0.65699377]
 [0.91286973]]
try decreasing
inp

...

minimum
-3.20892798947353
-0.64594835
0.02073708861384109
h1 change in forward
[[0.12294964 0.02073709]
 [0.23671011 0.10517288]]
outchange in forward
[[0.56987348]
 [0.84999159]]
after fwd
[[0.09215686 0.36078431 0.55294118]
 [0.29215686 0.46078431 0.55294118]]
[[0.12294964 0.02073709]
 [0.23671011 0.10517288]]
[[0.56987348]
 [0.84999159]]
before bkw
[[0.09215686 0.36078431 0.55294118]
 [0.29215686 0.46078431 0.55294118]]
[[0.12294964 0.02073709]
 [0.23671011 0.10517288]]
[[0.56987348]
 [0.84999159]]
[-0.93099414]
[-2.38419427]
[-0.93099414]
[-2.38419427]
[-2.09375105 -0.38513873]
[-15.48114204 -13.16956121]
[-2.09375105 -0.38513873]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.25986801 0.13479869]
 [1.         1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [3.94575692e-01 2.62873406e-01]]
[[0.56987348]
 [0.84999159]]
try increasing
input interval
[[0.09215686 0.36078431 0.55294118]
 [0.29215686 0.46078431 0.55294118]]
input before fwd
input
[[0.09215686 0.36078431 

h1 change in forward
[[1.99917929e-03 2.26586040e-04]
 [1.90201508e-01 9.97884852e-01]]
outchange in forward
[[5.34145219e-04]
 [9.34096874e-01]]
after fwd
[[0.         0.40326231 0.29410897]
 [0.90428339 1.         1.        ]]
[[1.99917929e-03 2.26586040e-04]
 [1.90201508e-01 9.97884852e-01]]
[[5.34145219e-04]
 [9.34096874e-01]]
consistency detected
try increasing
input interval
[[0.1        0.40326231 0.29410897]
 [0.90428339 1.         1.        ]]
try decreasing
input interval
[[0.1        0.40326231 0.29410897]
 [1.00428339 1.         1.        ]]
input before fwd
input
[[0.1        0.30326231 0.29410897]
 [1.00428339 1.         1.        ]]
[[0. 0.]
 [1. 1.]]
[[0.]
 [1.]]
minimum
-7.872457730770112
1.6656325
0.002011571860831458
h1 change in forward
[[0.00201157 0.        ]
 [0.34053622 1.        ]]
minimum
-7.354622049710316
-0.64594835
0.00033515896534111906
h1 change in forward
[[2.01157186e-03 3.35158965e-04]
 [3.40536216e-01 9.98950432e-01]]
outchange in forward
[[2.1335081

h1 change in forward
[[0.2687506  0.        ]
 [0.27119135 1.        ]]
minimum
-3.4512931702183742
-0.64594835
0.01634679542898211
h1 change in forward
[[0.2687506  0.0163468 ]
 [0.27119135 0.06335119]]
outchange in forward
[[0.61118516]
 [0.70958359]]
after fwd
[[0.05294118 0.3372549  0.54117647]
 [0.25294118 0.3372549  0.54117647]]
[[0.2687506  0.0163468 ]
 [0.27119135 0.06335119]]
[[0.61118516]
 [0.70958359]]
before bkw
[[0.05294118 0.3372549  0.54117647]
 [0.25294118 0.3372549  0.54117647]]
[[0.2687506  0.0163468 ]
 [0.27119135 0.06335119]]
[[0.61118516]
 [0.70958359]]
[-1.77216673]
[-2.21323254]
[-1.77216673]
[-2.21323254]
[-2.21383245 -0.48479617]
[-15.48114204 -13.16956121]
[-2.21383245 -0.48479617]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.27523147 0.15112737]
 [1.         1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [3.66282132e-01 2.44023729e-01]]
[[0.61118516]
 [0.70958359]]
try increasing
input interval
[[0.05294118 0.3372549  0.54117647]
 [0.252941

[-1.91124814 -0.23778368]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.23651821 0.11043714]
 [1.         1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [4.38902929e-01 2.92405007e-01]]
[[0.50337268]
 [0.90147778]]
try increasing
input interval
[[0.05294118 0.3372549  0.54117647]
 [0.25294118 0.5372549  0.74117647]]
input before fwd
input
[[0.05294118 0.3372549  0.54117647]
 [0.25294118 0.5372549  0.84117647]]
[[0. 0.]
 [1. 1.]]
[[0.]
 [1.]]
minimum
-4.248594557041047
1.6656325
0.07024303473546264
h1 change in forward
[[0.07024303 0.        ]
 [0.27119135 1.        ]]
minimum
-6.1504620430516255
-0.64594835
0.0011165323087480863
h1 change in forward
[[0.07024303 0.00111653]
 [0.27119135 0.11173246]]
outchange in forward
[[0.50337268]
 [0.90299586]]
after fwd
[[0.05294118 0.3372549  0.54117647]
 [0.25294118 0.5372549  0.84117647]]
[[0.07024303 0.00111653]
 [0.27119135 0.11173246]]
[[0.50337268]
 [0.90299586]]
before bkw
[[0.05294118 0.3372549  0.54117647]
 [0.25294118 

[-1.99911552 -0.30808222]
[-15.48114204 -13.16956121]
after bkw
[[0.         0.24776014 0.12209452]
 [1.         1.         1.        ]]
[[1.00000000e-06 1.00000000e-06]
 [4.17393389e-01 2.78074965e-01]]
[[0.5358038 ]
 [0.87403903]]
try increasing
input interval
[[0.07254902 0.37254902 0.59215686]
 [0.37254902 0.47254902 0.69215686]]
input before fwd
input
[[0.07254902 0.37254902 0.59215686]
 [0.37254902 0.47254902 0.79215686]]
[[0. 0.]
 [1. 1.]]
[[0.]
 [1.]]
minimum
-3.7385675938106044
1.6656325
0.11175534972886565
h1 change in forward
[[0.11175535 0.        ]
 [0.22093388 1.        ]]
minimum
-5.462319893930473
-0.64594835
0.0022194626240423935
h1 change in forward
[[0.11175535 0.00221946]
 [0.22093388 0.13088485]]
outchange in forward
[[0.5358038 ]
 [0.87762464]]
after fwd
[[0.07254902 0.37254902 0.59215686]
 [0.37254902 0.47254902 0.79215686]]
[[0.11175535 0.00221946]
 [0.22093388 0.13088485]]
[[0.5358038 ]
 [0.87762464]]
before bkw
[[0.07254902 0.37254902 0.59215686]
 [0.37254902 

In [132]:


k = 0
            test_val = 0
            while(test_val==0):
                print("testing 1")
                print("input interval")
                print(input_interval)
                test_interval = np.copy(input_interval)
                test_interval[0,k] = test_interval[0,k] - tol
                if(test_interval[0,k]<=0):
                    break
                test_val = test_consistency(input_interval)
                if(test_val== 0):
                    input_interval = np.copy(test_interval)
                else:
                    input_interval[0,k] = input_interval[0,k] + tol
                    break

                #now try increasing the upper bound
                print("try increasing")
                print("input interval")
                print(input_interval)
                test_interval = np.copy(input_interval)
                test_interval[1,k] = test_interval[1,k] + tol
                if(test_interval[1,k]>=1):
                    break
                test_val = test_consistency(input_interval)
                if(test_val==0):
                    input_interval = np.copy(test_interval)
                else:
                    input_interval[1,k] = input_interval[1,k] - tol
                    break
                k = (k + 1)%2
            print("rule generate")
            print(input_interval)
            #rules[t] = input_interval
            #t = t+1
            #print(rules)

IndentationError: unexpected indent (<ipython-input-132-71428b21e63b>, line 4)